### Introduction

The objective of the current study is to:

1. Conduct data wrangling
To this end, we will transform the data and obtain additional fields including
* utm_source
* utm_medium
* path.

2. Conduct data streaming

We will aggregate the data and for every 1000 records, obtain:

* min date - the smallest date in the batch
* max date - the largest date in the batch
* unique users - the number of unique users in the batch


#### Import data

In [21]:
import os
os.listdir("data/")

['Data Streaming.csv', 'data.csv', 'path_medium_source.csv']

In [22]:
import pandas as pd
df = pd.read_csv("data/data.csv")

In [23]:
#data overview
df.head()

,anonymous_user_id,url,time,browser,os,screen_resolution
0,b527ad6c-1e79-4ae7-8206-2bf4d127ec25,https://preview.hs-sites.com/_hcms/preview/tem...,1523551855,Chrome,Linux,800x600
1,2847753f-df48-4367-98dc-9028f6330532,https://www.cultureamp.com/employee-feedback/?...,1523549796,Chrome,Chrome OS,1366x768
2,7540e6a6-bd75-4f2b-b774-6309bda47c4f,https://www.cultureamp.com/,1523549868,Chrome,Mac OS X,2560x1440
3,db016ab6-daa3-45fb-84e6-7e6e4e1541ac,https://www.cultureamp.com/products/employee-e...,1523549913,Chrome,Mac OS X,1440x900
4,864557aa-10c5-4efa-a53e-33f7b8f1bd3c,https://www.cultureamp.com/employee-feedback/,1523548907,Chrome,Windows 8.1,1366x768


In [24]:
'''Extract path from the url'''
from urllib.parse import urlparse as up 

a = up("http://127.0.0.1:8000/Users/YodhResearch/Desktop/LongCtrl10min.tiff#anyfragment") 
paths = [up(item).path for item in df.url]

In [25]:
#add the new path variable
df['Path'] = paths

##### Examine the number of urls with utm

In [26]:
print("Number of urls with utm source:",df.url.str.contains("utm_source").sum())

Number of urls with utm source: 57857


In [27]:
print("Number of urls with utm medium:",df.url.str.contains("utm_medium").sum())

Number of urls with utm medium: 54120


In [28]:
print("Number of urls with utm campaign:",df.url.str.contains("utm_campaign").sum())

Number of urls with utm campaign: 53947


#### Extract Medium

For every url with a medum, there is pattern i.e., "(?P<utm_medium>(?<=utm_medium=).*?(?=&|\\s|$))" which points to the mdium. Therefore, we use re.findall to get the contents that match the specified pattern.

In [29]:
import re

In [30]:
'''Utility function to get medium'''
regex = r"(?P<utm_medium>(?<=utm_medium=).*?(?=&|\\s|$))"

def get_medium(url):
    medium = re.findall(regex, url)
    if len(medium) > 0:
        medium = medium[0]
    else:
        medium = "No Medium"
    return medium

#Get source using the get_source function
df['Medium'] = [get_medium(item) for item in df.url]

#### Extract Source

For every url with a medum, there is pattern i.e., "(?P<utm_source>(?<=utm_source=).*?(?=&|\\s|$))" which points to the source. Similalr to obtaining the medium, we will use re.findall to get the contents that match the specified pattern. If thr length of the source is greater than 0, then return the source, otherwise, no source was found and hence return "No source".

In [31]:
'''Utility function to get source'''
regex = r"(?P<utm_source>(?<=utm_source=).*?(?=&|\\s|$))"

def get_source(url):
    source = re.findall(regex, url)
    if len(source) > 0:
        source = source[0]
    else:
        source = "No source"
    return source

#Get source using the get_source function
df['Source'] = [get_source(item) for item in df.url]

In [32]:
df.head()

,anonymous_user_id,url,time,browser,os,screen_resolution,Path,Medium,Source
0,b527ad6c-1e79-4ae7-8206-2bf4d127ec25,https://preview.hs-sites.com/_hcms/preview/tem...,1523551855,Chrome,Linux,800x600,/_hcms/preview/template/multi,No Medium,No source
1,2847753f-df48-4367-98dc-9028f6330532,https://www.cultureamp.com/employee-feedback/?...,1523549796,Chrome,Chrome OS,1366x768,/employee-feedback/,cpc,google
2,7540e6a6-bd75-4f2b-b774-6309bda47c4f,https://www.cultureamp.com/,1523549868,Chrome,Mac OS X,2560x1440,/,No Medium,No source
3,db016ab6-daa3-45fb-84e6-7e6e4e1541ac,https://www.cultureamp.com/products/employee-e...,1523549913,Chrome,Mac OS X,1440x900,/products/employee-engagement/,No Medium,No source
4,864557aa-10c5-4efa-a53e-33f7b8f1bd3c,https://www.cultureamp.com/employee-feedback/,1523548907,Chrome,Windows 8.1,1366x768,/employee-feedback/,No Medium,No source


Lastly, we can save the new data with both source, medium, and path to a dataframe.

In [33]:
df.to_csv("data/path_medium_source.csv")

### Data Streaming

During data streaming we:

1. Obtain the date from the provded timie stamp using datetime

2. Aggregate the data using a step size of 1000 and get the maximum and minimum dates

3. Aggregate the data and obtain the number of unique users using the anonymous users id (ymous_user_id)

#### Obtain date

In [34]:
from datetime import datetime
#save to list
dates = []
for item in df.time:
    #get date
    date = datetime.fromtimestamp(item)
    dates.append(date)
    
df['Date'] = dates
#Minimum date

print("Smallest Date:" ,df.Date.min())
#largest date
print("Largest Date:" ,df.Date.max())

Smallest Date: 2018-04-01 03:03:09
Largest Date: 2018-04-15 02:59:49


##### Summary for every 1000 records

##### Maximum date

In [35]:
import numpy as np
x = df.Date
steps = 1000
#store position
pos = []
#store maximum date
max_dates = []
for i in range(0, len(x), steps):
    maximum = np.max(x[i:i+steps])
    position = i
    maxx = maximum
    #append
    max_dates.append(maxx)
    pos.append(position)
    
#add to dataframe

dfx = pd.DataFrame()
#Position
dfx['Position'] = pos
dfx['Maximum Date'] = max_dates


##### Minimum date

In [36]:
import numpy as np
x = df.Date
steps = 1000
#store position
pos = []
#store minimum date
min_dates = []
for i in range(0, len(x), steps):
    minimum = np.min(x[i:i+steps])
    position = i
    minx = minimum
    #append
    min_dates.append(minx)
    pos.append(position)
    
#add to dataframe

#Position
dfx['Position'] = pos
dfx['Minimum Date'] = min_dates


##### Number of unique users

In [37]:
import numpy as np
x = df.anonymous_user_id
steps = 1000
#store position
pos = []
#store minimum date
num_users = []
for i in range(0, len(x), steps):
    minimum = len(np.unique(x[i:i+steps]))
    position = i
    minx = minimum
    #append
    num_users.append(minx)
    pos.append(position)
    
#add to dataframe

#Position
dfx['Position'] = pos
dfx['Number of unique Users'] = num_users
#save the data to a dataframe
dfx.to_csv("data/Data Streaming.csv")
dfx

,Position,Maximum Date,Minimum Date,Number of unique Users
0,0,2018-04-12 19:57:12,2018-04-05 00:00:45,773
1,1000,2018-04-13 04:58:12,2018-04-05 00:00:08,788
2,2000,2018-04-09 19:57:11,2018-04-03 07:00:05,648
3,3000,2018-04-12 00:57:09,2018-04-09 04:01:41,766
4,4000,2018-04-11 22:58:06,2018-04-09 03:00:18,760
...,...,...,...,...
85,85000,2018-04-13 23:59:54,2018-04-01 06:00:36,808
86,86000,2018-04-14 01:59:48,2018-04-01 15:00:58,833
87,87000,2018-04-14 03:59:49,2018-04-01 17:00:48,790
88,88000,2018-04-15 02:59:49,2018-04-01 04:57:20,846
